In [ ]:
import sys
sys.path.insert(0,  "C:\\Users\\Paul Valcke\\Documents\\GitHub\\CHIMES" )
import chimes as chm


from IPython.display import display,HTML,Markdown
from IPython.display import IFrame
from itables import init_notebook_mode,options
options.columnDefs = [{"className": "dt-left", "targets": "_all"}]
options.classes="display nowrap compact"
options.scrollY="400px"
options.scrollCollapse=True
options.paging=False

init_notebook_mode(all_interactive=True)
%matplotlib widget

# Pandas display
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option("display.colheader_justify","left")

## Tomas Mazak's workaround
import plotly
import plotly.graph_objs as go
from IPython.display import display, HTML
plotly.offline.init_notebook_mode()
display(HTML('<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'))

# TL:DR (Too long, didn't read)

Here is the resume of everything seen in this tutorial
```
# TO KNOW WHAT YOU CAN LOAD
chm.get_available_models() # to know model names 
chn.get_available_saves() # to know saves names

# TO LOAD A MODEL
hub=chm.Hub('modelname',)
hub=chm.load_saved('savename or full adress')

hub.get_summary()           # Basic general informations
values = hub.get_dvalues()  # Values 

hub.run()                   # do the simulation
hub.plot()                  # return the results

hub.set_fields()            # change fields values
hub.set_preset(presetname)  # load a preset of values
hub.plot_preset()           # plot the preset content
```

# CHIMES Hub Basics

Models are made to interact with. All the interactions can be done through a CHIMES object called 'Hub`. Here, we show how to use it with one model, a Lorenz System (the famous chaos butterfly). 

## What is a Hub ?

The normal approach is to construct a Hub around a model. It is a CHIMES object that will decypher the model file and give you methods to interact with it. 

When you initialize a Huh with a model, it will: 
* Find what are all the fields that exist in the model
* Determine if they are differential, state variable, parameters
* Try to find as many existing information as possible (typically in CHIMES library) that has not been put by the model creator but exist in a common database. It can be :
    * units 
    * symbol
    * definition 
    * default value
    * ...
* Find an order to calculate the system at each timestep, and what variables are necessary (in loops)
* Preparing allocation for future time calculation and make calculation faster
* Making the model elements ready to be displayed in a user-friendly manner
* A bit of coffee for you 

In this example we call the created object `hub` (although the name could be anything you like !)


Two ways to create a Hub: 
* Creating it from a model file `hub=chm.Hub('modelname')` you can find models in `chm.get_available_models()` to get modelnames
* Loading a saved hub `hub=chm.Load('savename')` you can find saved files in `chm.get_available_saves()` to get savenames

### Knowing what is accessible to go in the hub

In [ ]:
modelist = chm.get_available_models(Return=True)
chm.get_available_models() # We check model names to be sure how we write it

In [ ]:
name = 'Lorenz_Attractor'
chm.get_model_documentation(name) # We look deeper to its properties

### Loading a model in a Hub

To load a model into the hub, simply declare the hub and the name of the model together `hub=chm.Hub(modelname)`

In [ ]:
hub=chm.Hub(name)   # For short !

### Getting basic informations 

In most situations, the method `get_summary()` should cover your needs. 
It covers the flags of the system, theiur preset names, and fields by category with their attributes. As we will see after, get_summary displays the last value of the simulation when a run has been done.

In [ ]:
hub.get_summary()           # Basic general informations

### Simulating 100 years

By default, simulations are done one 100 years, with a timestep of 0.1 year. When asking for a run, the system will compute each timestep using an RK4 scheme. The values will be updated in the `get_summary`

In [ ]:
hub.run()                   # do the simulation

### Representing all variables evolution

The easiest way to represent a run of the model is to display the variables. the function `hub.plot()` is doing that directly, with a classification of each variable by their unit 

In [ ]:
hub.plot()                  # return the results

### Retrieving values as a dictionnary

If you want to analyze the data by yourself, or plug then somewhere else, you can use `hub.get_dvalue` to get a dictionnary of the time evolution. A more extensive acces can be done with `get_dfields` as explained later

In [ ]:
values = hub.get_dvalues(params=True)  # Values 
print('type',type(values))
for k,v in values.items():
    print(k,np.shape(v))

### Changing a field value

The method `set_fields` is central in your interactions with the Hub. 
You can change fields that are differential variables (in this case, the initial condition), parameters, and numerical parameters. 
Of course you cannot impose state variables, but you can change the values of the variables it depends of

You can change either values one by one with `hub.set_fields(fieldname=value)`, or multiple at once `hub.set_fields(**{field1:value1, field2:value2, [...]})` 

it is always recommended to verify that it works using `hub.get_summar()` afterward

When a new value is set, the system automatically reset, if you want to save your previous run either do a `hub2=hub.copy()` or `hub.save(filename)` to keep a track of it.
You can change values in the middle of a run, but that's for advanced uses !

In [ ]:
hub.set_fields(Tsim=30) # The simulation now runs for 30 years
hub.set_fields(dt=0.04) # Timestep are now 0.05 years 
hub.set_fields(**dict(x=0,y=1,z=0))

hub.get_summary()

### Loading a preset

A preset is an ensemble of field values and plots that has been created to show a typical behavior of the system, or a calibrated run. 
You can find available preset using `hub.get_preset()` and then load it using `hub.set_preset`. 
Once the system has run, you can plot the preselected plots using `hub.plot_preset`

In [ ]:
hub.set_preset('Canonical example')
hub.run()
hub.plot_preset()

### Supplements

Some models have supplements, functions and method tailored for the model to manipulate it: finding analytically the equilibrium, doing a serie of plots... 
To acces the use `hub.get_available_supplements()`, they can be then used with `hub.supplements[supplementname](arguments_of_the_function)`

In [ ]:
hub=chm.Hub('Lorenz_Attractor')
hub.get_supplements()

In [ ]:
hub.supplements['OneTenthPercentUncertainty'](hub)
# In this example, the system run 10 times in parrallel with 1% of uncertainty on the initial conditions and parameters values

### Changing values :a first exposition to set_fields
`set_fields` is a very powerful tool to modify your system. 
Here is simply some basic uses.
* To change the simulated time, use `hub.set_fields(Tsim=YOUR_VALUE)`, it works with:   
    * 'Tsim' duration of the simulated time
    * 'dt'   duration of one timestep
    * 'nx'   number of systems in parallel
    * 'nr'   number of regions
* To change a parameter or an initial condition F, use `hub.set_fields(F=YOUR_VALUE)`
* To change multiple at a time, you can add each entry separately
* If you have multiple dimensions (parrallel, regions, multiple agents)... The system will be default broadcast, or adapt the value. 

In [ ]:
hub=chm.Hub('GK') 
hub.set_fields(Tsim=30) # The simulation now runs for 30 years
hub.set_fields(
    alpha=0,
    n=0.1
)
hub.get_summary()

Much more elaborated value settings can be done

In [ ]:
hub=chm.Hub('GK')
hub.set_fields(nx=6,nr=['France','China','USA'])         # We create many regions and parrallel system
hub.set_fields('nu',[['nr',0],['nx',0,4],[0.5,0.2]])     # We only modify one of those, with a list approach
print(hub.get_dfields()['nu']['value'][:,:,0,0])         # Here are the final values


hub.set_fields('alpha',{'nr':['France','USA'], 'nx':1, 'value':0.5}) #will do the same !
print(hub.get_dfields()['alpha']['value'][:,:,0,0])         # Here are the final values


### Saved files management

While it is not long to generate a simulation, it might be practical to save the result of one and load it later. 
You can do that as long as you use the same python version when you save and load, using `hub.save()` and `hub=chm.load_saved()`
See what files are available with `chm.get_available_saves()`, and specify a folder with `chm.get_available_saves(FOLDERNAME)`


In [ ]:
help(hub.save)

In [ ]:
help(chm.load_saved)

# Additional methods

#### Extract_preset

You have run your model, 

## calculate_ConvergeRate 

## calculate_Cycles

## copy

## dargs

## get_dvalues

## get_fieldsproperties

## get_new_summary

## reinterpolate_dfields

## reset

## set_dpreset

## short
